In [1]:
import requests
import pandas as pd
import numpy as np
from keys import *

city = "Paris"
country = "FR"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

In [2]:
data = response.json()

In [3]:
# Extract forecast list
forecast_list = data.get('list', [])

In [4]:
# Prepare lists for DataFrame columns
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

In [5]:
# Create DataFrame
df = pd.DataFrame({
    'weather_datetime': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind': wind_speeds,
    'rain_qty': rain_volumes,
    'snow': snow_volumes,
    'municipality_iso_country': f"Paris,FR"
})

print(df.head())

      weather_datetime  temperature  humidity weather_status  wind  rain_qty  \
0  2025-10-30 00:00:00        11.08        92         Clouds  4.63       NaN   
1  2025-10-30 03:00:00        11.19        81           Rain  2.99      0.11   
2  2025-10-30 06:00:00         8.63        73         Clouds  1.85       NaN   
3  2025-10-30 09:00:00        10.94        63         Clouds  2.33       NaN   
4  2025-10-30 12:00:00        13.22        47         Clouds  3.39       NaN   

   snow municipality_iso_country  
0   NaN                 Paris,FR  
1   NaN                 Paris,FR  
2   NaN                 Paris,FR  
3   NaN                 Paris,FR  
4   NaN                 Paris,FR  


In [6]:
import sqlalchemy
import pymysql
# connection details for the local mysql database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "ElMckenzie11$$$"
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [7]:
# send the weather data to the database
df.to_sql('weather_data', if_exists = 'append', con = con, index=False)

40